# Capture Object Movement 

In [ ]:
import cv2
import numpy as np
# enable web camera
cap = cv2.VideoCapture(0)
algo = cv2.createBackgroundSubtractorMOG2()
# algo = cv2.bgsegm.createBackgroundSubtractorMOG()
try:
    while True:
            ret, frame1 = cap.read()
#             This COLOR_BGR2LUV also give same result
#             gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2LUV)
            gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            blur = cv2.GaussianBlur(gray,(11,11),1)
            # Applying on each frame
            img_sub =  algo.apply(blur)
            thresh = cv2.threshold(img_sub,90,255,cv2.THRESH_BINARY)[1]
            kernal = np.ones((5, 5), np.float32)
            dilte = cv2.dilate(thresh,kernal,iterations=2)
            cv2.imshow('Other moving Objects ',dilte)

            cnt,h = cv2.findContours(dilte,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
            for i in cnt:
                if cv2.contourArea(i)<5000:
                    continue
                (x,y,w,h) = cv2.boundingRect(i)
                cv2.rectangle(frame1, (x,y),(x+w, y+h),(0,255,255),2)
            cv2.imshow("Detected", frame1)
            if cv2.waitKey(1)==27:
                break
    cv2.destroyAllWindows()
    cap.release()
except Exception as e:
    print("Caught error",e)
    cv2.destroyAllWindows()
    cap.release()

# Another Test 

In [21]:
import cv2,copy
import numpy as np
# enable web camera
cap = cv2.VideoCapture(0)
algo = cv2.createBackgroundSubtractorMOG2()
# algo = cv2.bgsegm.createBackgroundSubtractorMOG()
try:
    while True:
            ret, frame = cap.read()
            frame = cv2.bilateralFilter(frame, 5, 50, 100)  # Smoothing
            frame = cv2.flip(frame, 1)  #Horizontal Flip
            
            #Background Removal
            bgModel = cv2.createBackgroundSubtractorMOG2(0, 50)
            fgmask = bgModel.apply(frame)
            kernel = np.ones((3, 3), np.uint8)
            fgmask = cv2.erode(fgmask, kernel, iterations=1)
            img = cv2.bitwise_and(frame, frame, mask=fgmask)
            
             # Skin detect and thresholding
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            lower = np.array([0, 48, 80], dtype="uint8")
            upper = np.array([20, 255, 255], dtype="uint8")
            skinMask = cv2.inRange(hsv, lower, upper)
            cv2.imshow('Threshold Hands', skinMask) 
             # Getting the contours and convex hull
                
            skinMask1 = copy.deepcopy(skinMask)
            contours, hierarchy = cv2.findContours(skinMask1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            length = len(contours)
            maxArea = -1
            if length > 0:
                for i in range(length):
                    temp = contours[i]
                    area = cv2.contourArea(temp)
                    if area > maxArea:
                        maxArea = area
                        ci = i
                    res = contours[ci]
                    (x,y,w,h) = cv2.boundingRect(temp)
                    cv2.rectangle(frame, (x,y),(x+w, y+h),(0,255,255),2)
                hull = cv2.convexHull(res)
                drawing = np.zeros(img.shape, np.uint8)
                cv2.drawContours(drawing, [res], 0, (0, 255, 0), 2)
                cv2.drawContours(drawing, [hull], 0, (0, 0, 255), 3)
                cv2.imshow("drawing", drawing)
                
            cv2.imshow("Detected", frame)
            if cv2.waitKey(1)==27:
                break
    cv2.destroyAllWindows()
    cap.release()
except Exception as e:
    print("Caught error",e)
    cv2.destroyAllWindows()
    cap.release()